!pip install editdistance

In [4]:
%load_ext autoreload
%autoreload 2

import os
import datetime as dt

import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

import torch
import torch.nn as nn

import models
import encoders
import decoders
from training import train, test, checkpoint, main_wandb, main_simple
from dataloaders import load_data, make_loaders, append_SOS


SEED = 27

np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
cuda


In [5]:
M = 8

data_params = {'dataset_path': '/scratch/users/udemir15/ELEC491/bassline_transcription/data/datasets/[28, 51]',
               'dataset_name': 'TechHouse_bassline_representations',
               'scale_type': 'min',
               'M': M}

X, titles = load_data(data_params)
X = append_SOS(X)

K = X.max()+1 # Number of classes, assumes consecutive [0,max] inclusive
sequence_length = X.shape[1]

print('Number of classes: {}\nSequence Length: {}'.format(K, sequence_length))
print('Number of data points: {}'.format(X.shape[0]))

Number of classes: 27
Sequence Length: 65
Number of data points: 4421


In [ ]:
X.shape

X = X[:8]

frequencies = torch.tensor(np.unique(X, return_counts=True)).to(device)
cross_entropy_weights = frequencies[1].max() / frequencies[1]
cross_entropy_weights /= cross_entropy_weights.sum()
print('Cross entropy weights:\n{}\n'.format(cross_entropy_weights))
cross_entropy_weights = torch.FloatTensor(1 / (frequencies[1] / frequencies[1].max())).to(device)
print(cross_entropy_weights)

#encoder = encoders.LSTMEncoder(**encoder_params)
#decoder = decoders.SimpleLSTMDecoder(**decoder_params)
#model = models.Seq2SeqLSTM(encoder, decoder, device).to(device)

encoder = encoders.GRUEncoder(**encoder_params)
decoder = decoders.SimpleGRUDecoder(**decoder_params)
model = models.Seq2SeqGRU(encoder, decoder, device).to(device)

project_name = 'seq2seq_gru_simple'

encoder = encoders.GRUEncoder(**encoder_params)
decoder = decoders.GRUDecoder(**decoder_params)
model = models.Seq2SeqGRU(encoder, decoder, device).to(device)

project_name = 'seq2seq_gru'

In [28]:
train_params = {'batch_size': 32,
               'N_epochs': 150,
               'lr': 5e-4,
               'teacher_forcing_ratio': 0.75}

decoder_hidden_size = 64

encoder_params = {'input_size': K,
                  'embedding_size': 12,
                  'hidden_size': 64,
                  'output_size': decoder_hidden_size}

decoder_params = {'output_size': K,
                  'embedding_size': encoder_params['embedding_size'],
                  #'encoder_hidden_size': encoder_params['hidden_size'],
                  'hidden_size': decoder_hidden_size}

params = {'data': data_params,
          'encoder': encoder_params,
          'decoder':decoder_params, 
          'training':train_params}

project_name = 'seq2seq_gru_attention'

encoder = encoders.BidirectionalGRUEncoder(**encoder_params).to(device)
decoder = decoders.GRUDecoderWithAttention(**decoder_params).to(device)

model = models.Seq2SeqGRUWithAttention(encoder, decoder, device)

print(model)
print('Number of parameters: {}\n'.format(sum([parameter.numel() for parameter in model.parameters()])))

criterion = nn.CrossEntropyLoss(reduction='mean')
optimizer = torch.optim.Adam(model.parameters(), lr=train_params['lr'])

train_loader, validation_loader, test_loader = make_loaders(X, train_params)

main_wandb(model, criterion, optimizer, device, train_loader, validation_loader, test_loader, params, project_name)
#main_simple(model, criterion, optimizer, device, train_loader, validation_loader, test_loader, params)

Seq2SeqGRUWithAttention(
  (encoder): BidirectionalGRUEncoder(
    (embedding): Embedding(27, 12)
    (rnn): GRU(12, 64, batch_first=True, bidirectional=True)
    (fc): Linear(in_features=128, out_features=64, bias=True)
  )
  (decoder): GRUDecoderWithAttention(
    (attention): Attention(
      (attn): Linear(in_features=91, out_features=64, bias=True)
      (v): Linear(in_features=64, out_features=1, bias=False)
    )
    (embedding): Embedding(27, 12)
    (rnn): GRU(39, 64, batch_first=True)
    (fc_out): Linear(in_features=103, out_features=27, bias=True)
  )
)
Number of parameters: 67776



  0%|          | 0/150 [00:00<?, ?it/s]


Before Training:
Test Loss: 3.3426, Test Accuracy: 0.0120

Sample:
tensor([ 8,  0, 18, 10,  2, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24,
        24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24,
        24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24,
        24, 24, 24, 24, 24, 24, 24, 24, 24,  7], device='cuda:0')



  1%|          | 1/150 [00:18<45:15, 18.23s/it]

Epoch: 0, train_loss: 2.310966, val_loss: 2.058720
Sample:
tensor([ 1, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26,
        26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26,
        26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26,
        26, 26, 26, 26, 26, 26, 26, 26, 26, 26], device='cuda:0')


...

100%|██████████| 150/150 [32:05<00:00, 12.84s/it]



After Training:


test_loss,3.34261
test_accuracy,0.01204
_runtime,1938
_timestamp,1622968735
_step,153
train_loss,1.30147
train_accuracy,0.53201
validation_loss,1.48734
validation_acc,0.45483
teacher_forcing_ratio,0.29122


test_loss,▁
test_accuracy,▁
_runtime,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_timestamp,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train_loss,█▇▅▄▄▃▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁
train_accuracy,▁▂▃▄▄▅▅▆▆▆▇▇▇▇▇▇▇█████▇▇▇▆▆▆▆▆▆▆▆▆▆▆▆▇▇▇
validation_loss,█▇▆▅▅▅▄▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▄▃▃▃▂▂▂▁▂▁▁▁▁▁▁▁▁▁
validation_acc,▄▅▄▅▅▅▄▃▂▂▃▂▂▂▁▁▂▂▂▂▂▂▃▄▅▅▅▆▆▇▇▇▇███████
teacher_forcing_ratio,█████████████████████▇▆▅▄▃▃▂▁▁▁▁▁▁▁▁▁▁▁▁


IndexError: tuple index out of range

In [ ]:
def display_attention(sentence, translation, attention):
    
    fig = plt.figure(figsize=(10,10))
    ax = fig.add_subplot(111)
    
    attention = attention.squeeze(1).cpu().detach().numpy()
    a
    cax = ax.matshow(attention, cmap='bone')
   
    ax.tick_params(labelsize=15)
    
    #x_ticks = [''] + ['<sos>'] + [t.lower() for t in sentence] + ['<eos>']
    #y_ticks = [''] + translation
     
    #ax.set_xticklabels(x_ticks, rotation=45)
    #ax.set_yticklabels(y_ticks)

    #ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    #ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

    plt.show()
    plt.close()

In [ ]:
bassline = next(iter(train_loader))[0].to(device)

In [ ]:
reconstruction, attention = reconstruct_bassline(model, bassline)

In [ ]:
i=6
display_attention(bassline[i], reconstruction[i], attention[i])

In [ ]:
out, att = model(trg, trg, 0, True)

In [ ]:
len(att)

In [ ]:
att[0].shape

In [ ]:
test_loss, test_acc = test(model, test_loader, criterion, device)
samples = model.sample()

for epoch in tqdm(range(train_params['N_epochs'])):

    train_loss, train_acc = train(model, train_loader, optimizer, criterion, device)
    validation_loss, validation_acc = test(model, validation_loader, criterion, device)
    
    model.decoder.update_teacher_forcing_ratio(decoder_params['teacher_forcing_ratio']/train_params['N_epochs'])
    
    if epoch+1 % 5:
        print('Epoch: {}, train_loss: {:.6f}, acc: {:.3f}'.format(epoch+1, train_loss, train_acc))

test_loss, test_acc = test(model, test_loader, criterion, device)
print('Test Loss After Training: {:.6f}'.format(test_loss))

In [ ]:
encoder_params = {'input_size': K,
                  'embedding_size': 16,
                  'hidden_size': 256,
                  'n_layers': 1}

train_params = {'batch_size': 32,
               'N_epochs': 150,
               'lr': 5e-4}

In [ ]:
input_size  =K
embedding_size = 12
enc_hidden_size = 128
dec_hidden_size = 100

model = encoders.BidirectionalGRUEncoder(input_size, dec_hidden_size, embedding_size, enc_hidden_size)

In [ ]:
train_loader, validation_loader, test_loader = make_loaders(X, train_params)

batch = next(iter(train_loader))[0]

In [ ]:
outputs, hidden = model(batch)

In [ ]:
att = decoders.Attention(enc_hidden_size, dec_hidden_size)